In [54]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
# from Feed import Feed
from typing import Iterator, Any, List

In [3]:
from Surfer import GetBrowser,RedditFeed,Feed

driver = GetBrowser()

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:07<00:00, 1.01MB/s]


In [55]:
def uuid():
    letters_digits = string.ascii_letters + string.digits
    return ''.join(random.choice(letters_digits) for i in range(30))

In [46]:
def get_script(fname:str):
  script = ''
  with open(f'./scripts/{fname}.js','r') as f:
    script = f.read()
  script = script.replace('\n','')
  def replacer(*args):
    acc = script
    for i,x in enumerate(args):
      acc = acc.replace('#'+str(i),x)
    return f'console.log(`{fname}`,JSON.parse(`{json.dumps(args)}`));'+acc
  return replacer
def run_script(name:str,verbose=True):
	def script(*args:str):
		script_string = get_script(name)(*args)
		if verbose:
			print('script_string: ',script_string)
		return driver.execute_script(script_string)
	return script

In [47]:
def random_scroll(
  mean = 0.5
	, stddev = 0.15
):
	return max(0, min(random.normalvariate(mean, stddev) , 1))

In [48]:
class RedditFeed(Feed):
	def __init__(self, driver,skip=True):
		super()
		self.name = 'Reddit Feed'
		# Store the WebDriver instance
		self.driver = driver
		if not skip:
			self.driver.get('https://www.reddit.com/')

		run_script('init_reddit')()

	def __iter__(self) -> Iterator[Any]:
		while True:
			if not run_script('check curser')():
				continue
			yield run_script('read curser')()
			run_script('reddit scroll next')(str(random_scroll()),str(random_scroll()))

	def click_post(self):
		id = uuid()
		run_script('mark curser')(id)
		# Find the DOM element to click on
		element = driver.find_element(By.CSS_SELECTOR,f'.{id}')

		# Get the dimensions of the element
		element_size = element.size
		element_location = element.location

		# Calculate the x and y coordinates of the random point in the upper center of the element
		x = element_location["x"] + element_size["width"] / 2
		y = element_location["y"] + random.randint(0, element_size["height"] / 2)

		# Move the mouse to the calculated coordinates and click
		actions = ActionChains(driver)
		actions.move_to_element_with_offset(element, x - element_location["x"], y - element_location["y"]).click().perform()


  

	def close(self):
		# Close the WebDriver instance
		self.driver.quit()

In [ ]:
rf = RedditFeed(driver)

In [ ]:
for x in rf:
  print(x)
  time.sleep(len(x) * 0.01)

In [ ]:
# decultter html
# Array.from(x.querySelectorAll('*')).map((x,i)=>{
#     const cl = Array.from(x.classList)
#     const to_remove = cl.filter(x=> !/^[a-z-]+$/.exec(x))
#     for (let cls of to_remove) {
#         x.classList.remove(cls)
#     }
#     if (x.className === ''){
#         x.removeAttribute('class')
#     }
# })